In this Notebook you will find a sentiment analysis for the covid19-tweets data package with vader.
It is my first "bigger" project, so if you've feedback or suggestions, feel free to reach out to me.

Used data: 
* https://www.kaggle.com/gpreda/covid19-tweets 
* https://www.kaggle.com/fernandol/countries-of-the-world 
* https://www.kaggle.com/okfn/world-cities

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


**Data modification**

In [ ]:
df = pd.read_csv("/kaggle/input/covid19-tweets/covid19_tweets.csv")
data = pd.DataFrame(df[["user_name", "user_location", "date", "text"]])
data.head()

In [ ]:
#Non Ascii data will at the user_location column will be deleted to make it easier
#All nan values will be replaced by unknown

#def datacleaning(data):
data.fillna("unknown", inplace = True)
data[data["user_location"].str.contains(r'[^\x00-\x7F]+')]
data.info()


In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
#Sentiment value only store able as string? TODO -> Solution, need an INT value
#def Sentimentanalysis(data)
sid = SentimentIntensityAnalyzer()

data["sentiment"] = ""

for index, row in data.iterrows():
        data.at[index, "sentiment"] = sid.polarity_scores(str(row["text"]))["compound"]

In [ ]:

data["user_region"] = ""
countries = pd.read_csv("../input/countries-of-the-world/countries of the world.csv")
countries = countries[["Country", "Region"]]

cities = pd.read_csv("../input/world-cities/world-cities.csv")
cities = cities[["name", "country"]]

cities["country"] = cities["country"].str.rstrip()
countries["Country"] = countries["Country"].str.strip()
data["user_location"] = data["user_location"].str.strip()

location = cities.join(countries.set_index("Country"), on = "country")
location.head()

In [ ]:
#Mapping of the region on the data dataframe to do region based evaluations 
#Created with the friendly help of ShivamSingla(@ssingla)
#Processing time ~ 1 hour for (data(rows 179108) & location(rows ~ 23019))

from tqdm.notebook import tqdm_notebook

def regionfinder(a):
    for item in location.itertuples():
        if item[1] in a:
            return item[3]
        elif item[2] in a:
            return item[3]
    return "unknown"   

tqdm_notebook.pandas()
data["region"] = data["user_location"].progress_apply(regionfinder)

data.head()


In [ ]:
#Saving the data as csv to make it easier to work with the evaluations
data.to_csv("Aufbereitete Daten")


**Evaluation**

I´ve decided to write the evaluation as an extra part because the region mapping on the data dataframe takes about 1 hour

In [ ]:
import matplotlib as mlp
import matplotlib.pyplot as plt

In [ ]:
df2 = pd.read_csv("../input/aufbereitete-daten/Aufbereitete Daten")

In [ ]:
#Evaluation of the mean sentiment per region

sentimentperregion= df2[["sentiment", "region"]].groupby("region").mean()
sentimentperregion[["sentiment"]].plot(kind = "bar", grid = True)

In [ ]:
#Evaluation of the mean sentiment change over time world wide
def Print(a):
    print(a)

sentimentovertime = df2[["date", "sentiment"]]
sentimentovertime["month"] = sentimentovertime["date"].apply(lambda x: str(x)[5:7])
sentimentovertime = sentimentovertime.drop(["date"], axis = 1)
sentimentovertime = sentimentovertime.groupby("month").mean()

sentimentovertime = sentimentovertime.dropna()
sentimentovertime[["sentiment"]].plot(figsize= (5, 10), kind = "line" ,grid = True)
